### Imports

In [2]:
# from pymongo import MongoClient
# from pymongo.errors import ConnectionFailure, CollectionInvalid
import pandas as pd
from pandas import json_normalize
import numpy as np
import requests
# from sqlalchemy import create_engine
import ipywidgets as widgets
import plotly.graph_objects as go
from IPython.core.display import display
from re import search
from plotly.subplots import make_subplots
import plotly.express as px
from functools import partial
from geopy.geocoders import Nominatim
import logging
# from sqlalchemy.engine.url import URL
# from sqlalchemy_utils import database_exists, create_database
from utils.mongo_conn import MongoConnect
from utils.postgre_conn import PostgresConnect

In [3]:
logging.basicConfig(filename='./logs/drugs_errors.log', level=logging.INFO)

### Creating a MongoDB colleciton

In [5]:
mongo_conn = MongoConnect('team_G_db_mongo', 'drugs_recalled')
collection = mongo_conn.connect()

### Requesting Data through API

In [6]:
try:
    success=True
    for skip_records in range(0,13000, 1000):
        url = 'https://api.fda.gov/drug/enforcement.json?api_key=jzBDRKRffIUITo0uKg9EEWqasttW2cDni2b7ncUB&limit=1000&skip={0}'.format(skip_records)
        response = requests.get(url)
        if response.status_code == 200:
            json_to_dump = response.json()['results']
            collection.insert_many(json_to_dump)
        else:
            success=False
            break
    if success:
        print('Data Dump to MongoDB Successful.')
    else:
        print('Bad Request: ', response.status_code)
except ConnectionError:
    print('Unable to connect to source!. Try again after some time.')
except CollectionInvalid:
    print('Unable to connect to Mongo! Check logs for info!')
    logging.error("Exception occurred at mongo_connection", exc_info=True)
except:
    print('Oh No, something went wrong! Check logs for info!')
    logging.error("Exception occurred at mongo_connection", exc_info=True)

In [11]:
drugs_recalled_df = json_normalize(collection.find())

### Data Cleaning

#### Drop columns if null percentage greater than given percentage

In [16]:
drugs_recalled_df.isnull().sum()

_id                                 0
country                             1
city                                1
address_1                           1
reason_for_recall                   0
address_2                       11205
product_quantity                 1484
code_info                           0
center_classification_date          3
distribution_pattern                0
state                             764
product_description                 0
report_date                         0
classification                      0
recalling_firm                      0
recall_number                       3
initial_firm_notification          94
product_type                        0
event_id                            0
termination_date                 3441
recall_initiation_date              0
postal_code                       765
voluntary_mandated                 12
status                              0
openfda.application_number      10386
openfda.brand_name              10311
openfda.gene

In [15]:
drugs_recalled_df = drugs_recalled_df.replace("", np.nan)

In [12]:
def drop_cols(df, null_percentage):
    '''The function drops the columns which have null percentage greater than given null percentage by the user,
    INPUTS:
        df = The dataframe from which columns need to be dropped
        null_percentage = A numerical figure provided by user which acts as a threshold for null_percenatge 
    OUTPUT
        The function returns True if all columns are dropped successfully else it returns False
    '''
    try:
        total_rows = len(df)
        for column in df.columns:
            if (df[column].isnull().sum() / total_rows)*100> null_percentage and column!='brand_name':
                df.drop(columns=column, inplace=True)
        return True
    except Exception as e:
        logging.error("Exception occurred at drop_cols", exc_info=True)
        return False
if drop_cols(drugs_recalled_df, 85):
    print('Columns Dropped Successfully')
else:
    print('Unable to drop columns check logs for more info')

Columns Dropped Successfully


In [17]:
drugs_recalled_df.shape

(12503, 38)

#### Retrieveing values from lists inside columns

In [19]:
 
col_list = [col for col in drugs_recalled_df.columns if 'openfda' in col]

for column in col_list:
    col = column.split('.')[1]
    drugs_recalled_df[col] = drugs_recalled_df[column].astype('str').apply(lambda x: x.replace("'", "").replace('[','').replace(']','') if x!='nan' else None)
    drugs_recalled_df.drop(columns = column, inplace=True)



In [ ]:
drugs_recalled_df.drop(columns = [ '_id', 'product_type'], inplace=True)#Dropping redundant columns

In [18]:
drugs_recalled_df[drugs_recalled_df['country'].isnull()]

,_id,country,city,address_1,reason_for_recall,address_2,product_quantity,code_info,center_classification_date,distribution_pattern,...,openfda.product_ndc,openfda.product_type,openfda.route,openfda.substance_name,openfda.rxcui,openfda.spl_id,openfda.spl_set_id,openfda.package_ndc,openfda.is_original_packager,openfda.unii
9939,608701cdef9db1b3b93b2a90,NaN,NaN,NaN,Presence of Foriegn Tablets/Capsules: A single...,NaN,"23,064 bottles","H48496, H48497; Exp 05/15",20140422,US Nationwide including Puerto Rico and US Vir...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
drugs_recalled_df['country'].fillna(list(drugs_recalled_df[drugs_recalled_df['recalling_firm']=='Pfizer Inc.']['country'])[0], inplace=True)
drugs_recalled_df['city'].fillna(list(drugs_recalled_df[drugs_recalled_df['recalling_firm']=='Pfizer Inc.']['city'])[0], inplace=True)
drugs_recalled_df['address_1'].fillna(list(drugs_recalled_df[drugs_recalled_df['recalling_firm']=='Pfizer Inc.']['address_1'])[0], inplace=True)

#### Converting Date columns

In [21]:
date_cols = [col for col in drugs_recalled_df.columns if 'date' in col][:-1]

In [22]:
for column in date_cols:
    drugs_recalled_df[column] = pd.to_datetime(drugs_recalled_df[column], format = '%Y%m%d')

#### Getting Labels out of the reason_for_recall column

In [23]:
drugs_recalled_df['reason_main'] = pd.Series(drugs_recalled_df['reason_for_recall'].str.split('[:;.]').str.get(0)).str.upper()

In [24]:
drugs_recalled_df['reason_description'] = drugs_recalled_df['reason_for_recall'].str.split('[:;.]').apply(lambda x: "".join(x[1:]))

#### Filling Blank State Records with Actual States

In [28]:
geolocator = Nominatim(user_agent="Google")
def get_state(x):
    geocode = partial(geolocator.geocode, language="es")
    loc = geocode(x)
    if loc is None:
        return loc
    else:
        state = loc[0].split(',')[-2].strip()
        if state.isnumeric():
            state = loc[0].split(',')[-3].strip()
        return state
city_list = drugs_recalled_df[drugs_recalled_df['state'].isnull()]['city'].unique()
state_dict={}
for city in city_list:
    state_dict[city] = get_state(city)
for city, state in state_dict.items():
    drugs_recalled_df.loc[(drugs_recalled_df['city']==city) & (drugs_recalled_df['state'].isnull()), 'state']= state

In [ ]:
drugs_recalled_df['state'].isnull().sum()

In [ ]:
drugs_recalled_df.drop(columns = ['reason_for_recall'], inplace=True)

#### Creating TimeDelta Column

In [ ]:
drugs_recalled_df['time_diff'] = drugs_recalled_df['termination_date']-drugs_recalled_df['recall_initiation_date']

In [ ]:
drugs_recalled_df['voluntary_mandated'] = drugs_recalled_df['voluntary_mandated'].str.upper()

### Connection to Postgres

In [ ]:
def createDB_table(connection_dict, table_name):
    '''The fucntion creates databse and table and inserts data in table.
    INPUTS:
    connection_dict: Containing following key values
        user: Name of the user
        password: User's password to connect user to the service
        host: IP address of the host
        port: port number of postgres service
        db_name: Name of the database to be created, if it does not exits.
    '''
    try:
        engine = create_engine(URL(**dbConnectionParamDict))
        if not database_exists(engine.url):
            print('Database does not exists! Creating Database.')
            create_database(engine.url)
            print('Database created successfully!')
        else:
            # Connect the database if exists.
            print('Database Exists! Connecting...')
            engine.connect()
            print('Connection Successfull!')
        print('Creating Table...')
        drugs_recalled_df.to_sql(table_name, con=engine, index=False, if_exists='replace')#Insert Data in table
        print('Table Created Successfully')
    except OperationError:
        print('Operation Error! check logs for info!')
        logging.error("Exception occurred at createDB_table", exc_info=True)
    except Exception as e:
        print('Something went wrong! check logs for more info!')
        logging.error("Exception occurred at createDB_table", exc_info=True)
    finally:
        if engine:
            engine.dispose()
    

In [ ]:
db_connection_dict = {    'drivername' : 'postgres',
    'host'      : '192.168.56.30',
    'port'      : '5432',
    'database'  : 'team_G_db',
    'username'  : 'dap',
    'password'  : 'dap'
}
createDB_table(db_connection_dict, 'drugs_recalled')

### Get Data From PostGreSQL

In [ ]:
def get_data(connection_dict, query):
    '''The fucntion creates databse and table and inserts data in table.
    INPUTS:
    connection_dict: Containing following key values
        user: Name of the user
        password: User's password to connect user to the service
        host: IP address of the host
        port: port number of postgres service
        db_name: Name of the database to be created, if it does not exits.
    '''
    try:
        engine = create_engine(URL(**dbConnectionParamDict))
        if not database_exists(engine.url):
            print('Database does not exists!, check connection string!')
            return None
        else:
            # Connect the database if exists.
            print('Database Exists! Connecting...')
            engine.connect()
            print('Connection Successfull!')
        print('Creating Table...')
        df = pd.read_sql_query(query, engine)
        if engine:
            engine.dispose()
        print('Query executed Successfully')
        return df
    except UndefinedColumn:
        print('Column mentioned in query does not exists! check logs for info!')
        logging.error("Exception occurred at createDB_table", exc_info=True)
    except Exception as e:
        print('Something went wrong! check logs for more info!')
        logging.error("Exception occurred at createDB_table", exc_info=True)

        
    

### Drugs Recalled wrt to States and Classification

In [ ]:
query = 'SELECT country, classification, state, recall_initiation_date FROM drugs_recalled;'
bar_line_df = get_data(db_connection_dict, query)

In [ ]:
if bar_line_df is not None:
    figure = go.FigureWidget(layout = dict(title='Overall Recall Trend across the years'))
    for classification in bar_line_df['classification'].unique():
        temp_df = bar_line_df[(bar_line_df['classification']==classification)]
        values = temp_df['report_date'].value_counts().reset_index().sort_values(by='index')
        trace = go.Scatter(mode = 'lines', x = values['index'].astype('str'), y = values['report_date'], name=classification)
        figure.add_trace(trace)
    figure.update_layout(xaxis={'title':'report_date'}, yaxis={'title':'Number of Dugs Recalled'})
    figure.show()
                                       

Insights:
Annually an average of 1171.5 of recalls are enforced by FDA out if which 925.5 of them belong to Class II hazardous classification that is the drugs may have adversible health consequences.

In March,2019 533 recalls were enforced by FDA out of which 471, nearly 89%(can be seen as a spike in red), were classified as Class I drugs i.e. the drug recalled had a high probability of causing adverse health consequences or even cause death.

In [ ]:
if bar_line_df is not None:
    bar_line_df['year'] = bar_line_df['report_date'].dt.year
    bar_line_df['report_date'] = pd.to_datetime(bar_line_df['report_date'].dt.strftime('%Y-%m'))
    bar_line_df = bar_line_df[bar_line_df['country']=='United States']
    figure_dict = {}



    for year in sorted(bar_line_df['year'].unique()):
        figure = go.FigureWidget(layout = dict(width=400, height=400, title=str(year)))
        for classification in bar_line_df['classification'].unique():
            temp_df = bar_line_df[(bar_line_df['classification']==classification)& (bar_line_df['year']==year)]
            values = temp_df['report_date'].value_counts().reset_index().sort_values(by='index')
            trace = go.Scatter(mode = 'lines', x = values['index'].astype('str'), y = values['report_date'], name=classification)
            figure.add_trace(trace)
        figure.update_layout(xaxis={'title':'report_date'}, yaxis={'title':'Number of Dugs Recalled'})
        temp_df_2 = bar_line_df[bar_line_df['year']==year][['state', 'classification']].value_counts()
        temp_df_2 = temp_df_2.reset_index()
        temp_df_2.rename(columns = {0:'Drugs_Recalled'}, inplace=True)
        fig = px.bar(temp_df_2, x = 'classification', y='Drugs_Recalled',
                     color ='state', barmode='group',
                     width=400, height=400
                    )
        figure_2 = go.FigureWidget(fig)
        figure_dict[year] = [figure, figure_2]

    for year, figure_list in figure_dict.items():
        display(widgets.HBox(figure_list))

Insights:

1. In January 2014, 622 Class II drugs enforcement recalls were issued by FDA, which were nearly twice the enforcement issued in previous month.
2. In March 2019, it was North Carolina where FDA enforced 461 recalls of Class I hazardous drug.
3. There has been a decline of nearly 55%(2019-2020) in the enforcements issued by FDA.

### Creating TreeMap Graph to understand reason of Recall by each State

In [ ]:
query = 'SELECT country, state, reason_main, reason_description FROM drugs_recalled;'
drugs_df = get_data(db_connection_dict, query)

In [ ]:
if drugs_df is not None:
    grouped_df = drugs_df.groupby(by=['country','state','reason_main', 'reason_description']).agg({'reason_main':'count'})
    grouped_df.rename(columns= {'reason_main':"Count"}, inplace=True)
    grouped_df.reset_index(inplace=True)
    grouped_df['reason'] = ['Reasons for Recall']*len(grouped_df)
    fig = px.treemap(grouped_df,
                     path=['reason','country','state','reason_main', 'reason_description'],
                     values='Count',
                     color='Count',
                     maxdepth=3,
                     color_continuous_scale = 'blues'
                    )
    fig.show()


Insights:
1. For North Carolina, the reason for recall enforced by FDA in March 2019, was presence of MICROBIAL CONTAMINATION in drugs.

2. FDA has issued 466 enforcements of recall in Ontario, Canada because of the presence of Penicillin Cross Contamination in drugs.

3. Top 5 states where recalls were enforced:

    1. Florida, USA
    
    2. New Jersey, USA
    
    3. Illinois, USA
    
    4. Tennesse, USA
    
    5. California, USA

4. Top 5 reasons for recall were:

    1. Lack of Assurance of Sterility
    2. CGMP Deviations
    3. Labelling
    4. Penicillin Cross Contamination
    5. Microbial Contamination



### Insight on Time taken by FDA to terminate report

In [ ]:
query = 'SELECT classification, time_diff FROM drugs_recalled;'
time_diff_df = get_data(db_connection_dict, query)

In [ ]:
time_diff_df['time_diff'] = ((pd.to_timedelta(time_diff_df['time_diff']))/np.timedelta64(1, 'M')).round()
time_diff_df = time_diff_df[~(time_diff_df['classification']=='Not Yet Classified')]

In [ ]:
print(time_diff_df.groupby('classification')['time_diff'].max())
print(time_diff_df.groupby('classification')['time_diff'].min())

Insights:
1. Since the initiation of recall, it took a maximum of 90 months by FDA to ensure all the reasonable efforts have been made to remove the product or correct it in accordance with recall strategy.
2. Class II, III recalls can be terminated in span of a month as well.

### Notification Means vs Current Status

In [ ]:
query = 'Select initial_firm_notification, status from drugs_recalled'
termination_status_df = get_data(db_connection_dict, query)

In [ ]:
termination_status_df = pd.DataFrame(termination_status_df.value_counts()).reset_index()
termination_status_df.rename(columns={0:'Count'}, inplace=True)
termination_status_df['initial_firm_notification'] = termination_status_df['initial_firm_notification'].str.replace('Two or more of the following: Email, Fax, Letter, Press Release, Telephone, Visit','More than Two Means')

In [ ]:
figure = go.Figure()
for status in termination_status_df['status'].unique():
    temp_df = termination_status_df[(termination_status_df['status']==status)]
    trace = go.Bar(x = temp_df['initial_firm_notification'], y = temp_df['Count'], name=status)
    figure.add_trace(trace)
figure.update_layout(title='Notification vs Status Count', xaxis=dict(title='Notification Method'), yaxis=dict(title='Recalls Reported'))


Insights:
1. Out of 9004 Terminated recalls, 42.6% of recalls were reported via Letters.